In [2]:
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import time
import pickle
import requests
import sys
sys.path.append('./scripts')
import data_download as dd

odds_api_key = '88e6315582eb26883c0fe52473dce025'
add_on = '_srs'
updating=False

In [3]:
if updating:
    res = dd.make_stats_data_table()
    res.to_csv('./Data/Stats/cur_stats_frame.csv', index=False)
    dd.get_week_inputs(all_season=True, update=True)

In [2]:
#dd.download_and_update_datasets()
#dd.get_week_inputs(all_season=True, update=False)#['2023-10-5','2023-10-6','2023-10-7','2023-10-8'])

In [2]:
#dd.update_dataframes(add_on)

2023-10-21 15:05:28.274745: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-21 15:05:28.328473: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-21 15:05:28.328535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-21 15:05:28.331569: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-21 15:05:28.331722: I tensorflow/compile

Index(['Games', 'pass_completion', 'pass_attempts', 'pass_completion_percent',
       'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard',
       'rush_td', 'num_plays', 'total_yard', 'yard_per_play',
       'pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs',
       'penalty', 'penalty_yards', 'lost_fumble', 'interceptions', 'turnovers',
       'SRS', 'SOS'],
      dtype='object')
Index(['Games', 'pass_completion', 'pass_attempts', 'pass_completion_percent',
       'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard',
       'rush_td', 'num_plays', 'total_yard', 'yard_per_play',
       'pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs',
       'penalty', 'penalty_yards', 'lost_fumble', 'interceptions', 'turnovers',
       'SRS', 'SOS'],
      dtype='object')
2013:0
2014:1362
2015:2780
2016:4208
2017:5623
2018:7066
2019:8524
2020:9982
2021:10934
2022:12379


In [4]:
whole_season = True
if not whole_season:
    cur_preds = dd.load_data('./Data/predictions/2023-10-02.pkl')
else:
    this_season = dd.load_data('./Data/predictions/all_season.pkl')
    cur_preds = this_season.where(this_season['Label'] == '')
    cur_preds = cur_preds.where(cur_preds['Binary_Label'] == 0)
    cur_preds = cur_preds.dropna()
    
columns_cur = dd.load_data(f'./Dataset/scaled_dataset{add_on}.pkl').columns.to_list()[1 if add_on == '' else 2:]

In [5]:
cur_preds.columns

Index(['Date', 'Team', 'Location', 'Opponent', 'Conference', 'Label',
       'Binary_Label', 'pass_completion', 'pass_attempts',
       'pass_completion_percent', 'pass_yards', 'pass_td', 'rush_attempt',
       'rush_yards', 'avg_rush_yard', 'rush_td', 'num_plays', 'total_yard',
       'yard_per_play', 'pass_first_down', 'rush_first_down', 'penalty_first',
       'first_downs', 'penalty', 'penalty_yards', 'lost_fumble',
       'interceptions', 'turnovers', 'SRS', 'SOS', 'op_pass_completion',
       'op_pass_attempts', 'op_pass_completion_percent', 'op_pass_yards',
       'op_pass_td', 'op_rush_attempt', 'op_rush_yards', 'op_avg_rush_yard',
       'op_rush_td', 'op_num_plays', 'op_total_yard', 'op_yard_per_play',
       'op_pass_first_down', 'op_rush_first_down', 'op_penalty_first',
       'op_first_downs', 'op_penalty', 'op_penalty_yards', 'op_lost_fumble',
       'op_interceptions', 'op_turnovers', 'op_SRS', 'op_SOS', 'Team_name',
       'Op_name'],
      dtype='object')

In [7]:
if whole_season:
    print(this_season)

            Date   Team  Location  Opponent  Conference Label  Binary_Label  \
0       2023-9-9    0.0         2      93.0           4    13             2   
1      2023-9-15    0.0         0     119.0           7    39             2   
2      2023-9-22    0.0         1      95.0           7    45             2   
3      2023-9-30    0.0         0      94.0           7    49             2   
4     2023-10-14    0.0         0     132.0           7    34             2   
...          ...    ...       ...       ...         ...   ...           ...   
1475  2023-10-28  132.0         1      12.0           7     7             1   
1476   2023-11-3  132.0         0      24.0           7                   0   
1477  2023-11-10  132.0         1      71.0           7                   0   
1478  2023-11-18  132.0         0      38.0           7                   0   
1479  2023-11-25  132.0         1      70.0           7                   0   

     pass_completion pass_attempts pass_completion_

In [8]:
columns_cur

['Team',
 'Location',
 'Opponent',
 'Conference',
 'pass_completion',
 'pass_attempts',
 'pass_completion_percent',
 'pass_yards',
 'pass_td',
 'rush_attempt',
 'rush_yards',
 'avg_rush_yard',
 'rush_td',
 'num_plays',
 'total_yard',
 'yard_per_play',
 'pass_first_down',
 'rush_first_down',
 'penalty_first',
 'first_downs',
 'penalty',
 'penalty_yards',
 'lost_fumble',
 'interceptions',
 'turnovers',
 'SRS',
 'SOS',
 'op_pass_completion',
 'op_pass_attempts',
 'op_pass_completion_percent',
 'op_pass_yards',
 'op_pass_td',
 'op_rush_attempt',
 'op_rush_yards',
 'op_avg_rush_yard',
 'op_rush_td',
 'op_num_plays',
 'op_total_yard',
 'op_yard_per_play',
 'op_pass_first_down',
 'op_rush_first_down',
 'op_penalty_first',
 'op_first_downs',
 'op_penalty',
 'op_penalty_yards',
 'op_lost_fumble',
 'op_interceptions',
 'op_turnovers',
 'op_SRS',
 'op_SOS']

In [9]:
def load_sequential_model(file_path):
    try:
        loaded_model = tf.keras.models.load_model(file_path)
        print("Model loaded successfully from:", file_path)
        return loaded_model
    except Exception as e:
        print("Error while loading the model:", str(e))
        return None

pred_model = load_sequential_model('./Models/binary_52_all.h5')

2023-11-02 20:57:36.154355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 20:57:36.212713: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 20:57:36.212758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 20:57:36.214385: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 20:57:36.214472: I tensorflow/compile

Model loaded successfully from: ./Models/binary_52_all.h5


In [10]:
def make_predictions(model, pred_data):
    #data = dd.load_data(f'./Data/predictions/2023-09-29.pkl')
    data = pred_data
    minmax = dd.load_data(f'./Dataset/MMScaler{add_on}.pkl')
    team_names = data[['Date', 'Team_name', 'Op_name']]
    team_names.columns = ['Date', 'Team_name', 'Op_name']
    data = data.drop(['Label', 'Binary_Label', 'Date','Team_name', 'Op_name'], axis=1)
    scaled_data = pd.DataFrame(minmax.transform(data[columns_cur]))
    scaled_data.columns = columns_cur
    inputs = scaled_data[columns_cur]
    predictions = model.predict(np.array(inputs, dtype=float))
    final_preds = team_names.copy()
    final_preds[['loss_pred', 'win_pred']] = predictions
    return final_preds.reset_index(drop=True)

predictions = make_predictions(pred_model, cur_preds)

16/16 [==============================] - 2s 4ms/step


In [11]:
predictions

,Date,Team_name,Op_name,loss_pred,win_pred
0,2023-11-4,air-force,army,0.086499,0.913501
1,2023-11-11,air-force,hawaii,0.035962,0.964037
2,2023-11-18,air-force,nevada-las-vegas,0.186737,0.813263
3,2023-11-24,air-force,boise-state,0.215198,0.784802
4,2023-11-8,akron,miami-oh,0.965743,0.034257
...,...,...,...,...,...
499,2023-11-25,wisconsin,minnesota,0.595663,0.404337
500,2023-11-3,wyoming,colorado-state,0.333283,0.666717
501,2023-11-10,wyoming,nevada-las-vegas,0.697046,0.302954
502,2023-11-18,wyoming,hawaii,0.098105,0.901895


In [10]:
#teams_pred = teams_pred.reset_index(drop=True)

#print(len([index for index, _ in teams_pred.iterrows()]))
used = []
rows = []
for row in predictions.iterrows():
    #index 0 is the index of the row and index 1 is the data in that row
    idx = row[0]
    if idx in used:
        continue
    data = row[1]
    mask = (predictions['Date'] == data['Date']) & (predictions['Op_name'] == data['Team_name']) & (predictions['Team_name'] == data['Op_name'])
    op_data = predictions[mask]
    op_idx = op_data.index[0]
    used.extend([idx, op_idx])
    team_win = float((data['win_pred'] + op_data['loss_pred'])/2)
    op_win = float((data['loss_pred'] + op_data['win_pred'])/2)
    rows.append(data.to_list()[:3] + [team_win, op_win])



/tmp/ipykernel_515664/3294754124.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  team_win = float((data['win_pred'] + op_data['loss_pred'])/2)
/tmp/ipykernel_515664/3294754124.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  op_win = float((data['loss_pred'] + op_data['win_pred'])/2)


In [11]:
rows

[['2023-10-28',
  'air-force',
  'colorado-state',
  0.9144428968429565,
  0.08555707335472107],
 ['2023-11-4', 'air-force', 'army', 0.9249862432479858, 0.07501375675201416],
 ['2023-11-11',
  'air-force',
  'hawaii',
  0.9575035572052002,
  0.04249637573957443],
 ['2023-11-18',
  'air-force',
  'nevada-las-vegas',
  0.8341078758239746,
  0.16589213907718658],
 ['2023-11-24',
  'air-force',
  'boise-state',
  0.8765060901641846,
  0.12349388003349304],
 ['2023-11-1', 'akron', 'kent-state', 0.4094606935977936, 0.590539276599884],
 ['2023-11-8', 'akron', 'miami-oh', 0.04584423452615738, 0.9541558027267456],
 ['2023-11-14',
  'akron',
  'eastern-michigan',
  0.16506701707839966,
  0.8349330425262451],
 ['2023-11-24', 'akron', 'ohio', 0.05996682494878769, 0.9400331974029541],
 ['2023-11-4',
  'alabama',
  'louisiana-state',
  0.618121862411499,
  0.381878137588501],
 ['2023-11-11', 'alabama', 'kentucky', 0.9074421525001526, 0.0925578624010086],
 ['2023-11-25', 'alabama', 'auburn', 0.902407

In [12]:
p_df = pd.DataFrame(rows)
print(p_df)
p_df.columns = ['Date','A', 'B', 'A_win', 'B_win']
p_df['ABS_diff'] = abs(p_df['A_win'] - p_df['B_win'])
p_df['prediction'] = np.where(p_df['A_win'] > p_df['B_win'], p_df['A'], p_df['B'])
if not whole_season:
    p_df.to_csv(f'./Data/predictions/{date.today()}_preds.csv', index=False)
else:
    p_df.to_csv(f'./Data/predictions/all_season_preds.csv', index=False)

              0                    1                 2         3         4
0    2023-10-28            air-force    colorado-state  0.914443  0.085557
1     2023-11-4            air-force              army  0.924986  0.075014
2    2023-11-11            air-force            hawaii  0.957504  0.042496
3    2023-11-18            air-force  nevada-las-vegas  0.834108  0.165892
4    2023-11-24            air-force       boise-state  0.876506  0.123494
..          ...                  ...               ...       ...       ...
301   2023-11-4  southern-california        washington  0.218821  0.781179
302  2023-11-11                 utah        washington  0.218608  0.781392
303   2023-11-4        texas-el-paso  western-kentucky  0.088662  0.911338
304  2023-11-25             virginia     virginia-tech  0.302030  0.697970
305  2023-11-25           washington  washington-state  0.861897  0.138103

[306 rows x 5 columns]


In [28]:
def get_college_football_odds(api_key):
    # Set up the API endpoint and parameters
    api_url = "https://api.the-odds-api.com/v3/odds"
    
    # Calculate the date range for the weekend
    today = datetime.now()
    start_date = today + timedelta(days=(5 - today.weekday()))  # Assuming today is Sunday (weekday 6)
    end_date = start_date + timedelta(days=2)  # Weekend includes Saturday and Sunday

    # Format dates as strings
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")

    # Define parameters
    parameters = {
        'apiKey': api_key,
        'sport': 'americanfootball_ncaaf',
        'region': 'us',
        'mkt': 'h2h',
        'dateFormat': 'iso',
        'oddsFormat': 'american',
        'startDate': start_date_str,
        'endDate': end_date_str,
    }

    # Make the request
    response = requests.get(api_url, params=parameters)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()
        return data['data']
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

In [29]:
college_football_odds = get_college_football_odds(odds_api_key)

In [51]:
college_football_odds

[{'id': '0cfa44874d9343bc9409ec62a9744f35',
  'sport_key': 'americanfootball_ncaaf',
  'sport_nice': 'NCAAF',
  'teams': ['Jacksonville State Gamecocks', 'Middle Tennessee Blue Raiders'],
  'home_team': 'Middle Tennessee Blue Raiders',
  'commence_time': '2023-10-05T00:00:00Z',
  'sites': [{'site_key': 'draftkings',
    'site_nice': 'DraftKings',
    'last_update': '2023-10-02T12:30:20Z',
    'odds': {'h2h': [150, -180]}},
   {'site_key': 'fanduel',
    'site_nice': 'FanDuel',
    'last_update': '2023-10-02T12:30:20Z',
    'odds': {'h2h': [134, -162]}},
   {'site_key': 'bovada',
    'site_nice': 'Bovada',
    'last_update': '2023-10-02T12:29:36Z',
    'odds': {'h2h': [155, -180]}},
   {'site_key': 'unibet_us',
    'site_nice': 'Unibet',
    'last_update': '2023-10-02T12:30:04Z',
    'odds': {'h2h': [135, -165]}},
   {'site_key': 'betrivers',
    'site_nice': 'BetRivers',
    'last_update': '2023-10-02T12:30:30Z',
    'odds': {'h2h': [135, -165]}},
   {'site_key': 'williamhill_us',
    

In [52]:
def extract_best_odds(response_data):
    odds_list = []

    for event in response_data:
        if 'sites' in event:
            teams = event['teams']
            site_with_best_odds = max(event['sites'], key=lambda x: x['odds']['h2h'][0])
            best_odds_a = site_with_best_odds['odds']['h2h'][0]
            best_odds_b = site_with_best_odds['odds']['h2h'][1]

            odds_list.append({
                'Team_a': teams[0],
                'Team_b': teams[1],
                'a_odds': best_odds_a,
                'b_odds': best_odds_b
            })

    return pd.DataFrame(odds_list)

In [53]:
best_odds = extract_best_odds(college_football_odds)

In [54]:
best_odds.to_csv(f'./Data/predictions/{date.today()}_odds.csv', index=False)